In [6]:

import os
os.chdir("../")

In [1]:
%pwd
# %pip install langchain
# %pip install langchain-community
# %pip install pypdf


'd:\\MedAccessAI\\research'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [10]:
extracted_data=load_pdf_file(data='Data/')

In [11]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [18]:
%pip install sentence-transformers

  Using cached transformers-4.50.0-py3-none-any.whl.metadata (39 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached huggingface_hub-0.29.3-py3-none-any.whl (468 kB)
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.2 MB ? eta -:--:--
   -------------------

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Asus PC\AppData\Local\Temp\ipykernel_23576\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [1]:
%pip install dotenv
%pip install pinecone



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')



In [19]:
from pinecone import Pinecone, ServerlessSpec
import os


# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name
index_name = "industrial"

# # Create the index
# pc.create_index(
#     name=index_name,
#     dimension=384, 
#     metric="cosine", 
#     spec=ServerlessSpec(
#         cloud="aws", 
#         region="us-east-1"
#     ) 
# )


In [20]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY



In [37]:
%pip install langchain-community pinecone-client


Note: you may need to restart the kernel to use updated packages.


In [20]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [21]:
from langchain.vectorstores import Pinecone
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [22]:
docsearch

In [23]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_docs = retriever.invoke("What is Acne?")
context = "\n".join([doc.page_content for doc in retrieved_docs])
print("Retrieved Context:", context)

Retrieved Context: GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26
GALE ENCYCLOPEDIA OF MEDICINE 2 25
Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceous
glands become inflamed.(Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25
Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when the
pores of the skin become clogged with oil, dead skin
cells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
the most common skin disease. It affects nearly 17 million
people in the United States. While acne can arise at any


In [25]:
retrieved_docs

[Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'creationdate': '2

In [25]:
# Use Groq API for text generation (LLM functionality)
import os
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [31]:
%pip install groq
%pip install requests


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [26]:

import requests
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from groq import Groq

# Initialize the client
client = Groq(
    api_key=GROQ_API_KEY,
)

def generate_text_groq(prompt, max_length=500):
    try:
        completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="llama-3.3-70b-versatile",  # or another available model
            max_tokens=max_length,
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating text: {str(e)}")
        raise

In [27]:
# Generate answers using retrieved context
question = "What is Acromegaly and gigantism?"
prompt = f"Use the following context to answer the question concisely:\n\n{context}\n\nQuestion: {question}\nAnswer:"
answer = generate_text_groq(prompt)
print("Answer:", answer)

Answer: The text doesn't mention Acromegaly and gigantism. It discusses Acne, its definition, and description.


In [28]:
question = "What is stats?"
prompt = f"Use the following context to answer the question concisely:\n\n{context}\n\nQuestion: {question}\nAnswer:"
answer = generate_text_groq(prompt)
print("Answer:", answer)

Answer: 17 million people in the United States are affected by acne.
